In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vietnamese-online-news-dataset/news_dataset.json


In [14]:
import json
import pandas as pd
import re
import regex
import unicodedata

# Đọc dữ liệu
with open("/kaggle/input/vietnamese-online-news-dataset/news_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Chuyển đổi sang DataFrame
news_df = pd.DataFrame(data)

# Lấy 1000 bài viết đầu tiên trong cột 'content'
contents = news_df['content'].dropna()

# Tách câu tiếng Việt
def split_sentences(text):
    text = regex.sub(r"([\p{L}\p{N}])([\.!?])(?=\s|$)", r"\1\2<SPLIT>", text)
    parts = text.split("<SPLIT>")
    return [part.strip() for part in parts if part.strip()]

# Bảng ánh xạ ký tự tiếng Việt sang không dấu
VN_DIACRITIC_MAP = {
    'à': 'a', 'á': 'a', 'ả': 'a', 'ã': 'a', 'ạ': 'a', 'ă': 'a', 'ắ': 'a', 'ằ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ặ': 'a', 'â': 'a', 'ấ': 'a', 'ầ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ậ': 'a',
    'À': 'A', 'Á': 'A', 'Ả': 'A', 'Ã': 'A', 'Ạ': 'A', 'Ă': 'A', 'Ắ': 'A', 'Ằ': 'A', 'Ẳ': 'A', 'Ẵ': 'A', 'Ặ': 'A', 'Â': 'A', 'Ấ': 'A', 'Ầ': 'A', 'Ẩ': 'A', 'Ẫ': 'A', 'Ậ': 'A',
    'đ': 'd', 'Đ': 'D',
    'è': 'e', 'é': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ẹ': 'e', 'ê': 'e', 'ề': 'e', 'ế': 'e', 'ể': 'e', 'ễ': 'e', 'ệ': 'e',
    'È': 'E', 'É': 'E', 'Ẻ': 'E', 'Ẽ': 'E', 'Ẹ': 'E', 'Ê': 'E', 'Ề': 'E', 'Ế': 'E', 'Ể': 'E', 'Ễ': 'E', 'Ệ': 'E',
    'ì': 'i', 'í': 'i', 'ỉ': 'i', 'ĩ': 'i', 'ị': 'i',
    'Ì': 'I', 'Í': 'I', 'Ỉ': 'I', 'Ĩ': 'I', 'Ị': 'I',
    'ò': 'o', 'ó': 'o', 'ỏ': 'o', 'õ': 'o', 'ọ': 'o', 'ô': 'o', 'ồ': 'o', 'ố': 'o', 'ổ': 'o', 'ỗ': 'o', 'ộ': 'o', 'ơ': 'o', 'ờ': 'o', 'ớ': 'o', 'ở': 'o', 'ỡ': 'o', 'ợ': 'o',
    'Ò': 'O', 'Ó': 'O', 'Ỏ': 'O', 'Õ': 'O', 'Ọ': 'O', 'Ô': 'O', 'Ồ': 'O', 'Ố': 'O', 'Ổ': 'O', 'Ỗ': 'O', 'Ộ': 'O', 'Ơ': 'O', 'Ờ': 'O', 'Ớ': 'O', 'Ở': 'O', 'Ỡ': 'O', 'Ợ': 'O',
    'ù': 'u', 'ú': 'u', 'ủ': 'u', 'ũ': 'u', 'ụ': 'u', 'ư': 'u', 'ừ': 'u', 'ứ': 'u', 'ử': 'u', 'ữ': 'u', 'ự': 'u',
    'Ù': 'U', 'Ú': 'U', 'Ủ': 'U', 'Ũ': 'U', 'Ụ': 'U', 'Ư': 'U', 'Ừ': 'U', 'Ứ': 'U', 'Ử': 'U', 'Ữ': 'U', 'Ự': 'U',
    'ỳ': 'y', 'ý': 'y', 'ỷ': 'y', 'ỹ': 'y', 'ỵ': 'y',
    'Ỳ': 'Y', 'Ý': 'Y', 'Ỷ': 'Y', 'Ỹ': 'Y', 'Ỵ': 'Y'
}

def remove_vietnamese_diacritics(text):
    return ''.join(VN_DIACRITIC_MAP.get(c, c) for c in text)

# Xử lý dữ liệu
processed_data = []
for idx, article in enumerate(contents):
    sentences = split_sentences(article)
    for sentence in sentences:
        no_diacritics = remove_vietnamese_diacritics(sentence)
        processed_data.append({
            "article_id": idx,
            "with_diacritics": sentence,
            "no_diacritics": no_diacritics
        })

# Lưu lại dữ liệu đã xử lý
processed_df = pd.DataFrame(processed_data)
processed_df.to_csv("processed_vietnamese_sentences.csv", index=False, encoding="utf-8-sig")


In [16]:
processed_df.head(5)

,article_id,with_diacritics,no_diacritics
0,0,"Chiều 31/7, Công an tỉnh Thừa Thiên - Huế đã c...","Chieu 31/7, Cong an tinh Thua Thien - Hue da c..."
1,0,"Sau đó, đối tượng mang số vàng vừa cướp được v...","Sau do, doi tuong mang so vang vua cuop duoc v..."
2,0,Giám đốc Công an tỉnh Thừa Thiên – Huế lập tức...,Giam doc Cong an tinh Thua Thien – Hue lap tuc...
3,0,Hàng trăm tiểu thương trong chợ Đông Ba và ngư...,Hang tram tieu thuong trong cho Dong Ba va ngu...
4,0,"Tuy nhiên, thấy vàng bị ném ra đường, nhiều ng...","Tuy nhien, thay vang bi nem ra duong, nhieu ng..."


In [15]:
len(processed_df)

3062231